# 第4章: 言語解析
問題30から問題35までは、以下の文章text（太宰治の『走れメロス』の冒頭部分）に対して、言語解析を実施せよ。問題36から問題39までは、国家を説明した文書群（日本語版ウィキペディア記事から抽出したテキスト群）をコーパスとして、言語解析を実施せよ。

https://nlp100.github.io/2025/ja/ch04.html

In [ ]:
text = """
メロスは激怒した。
必ず、かの邪智暴虐の王を除かなければならぬと決意した。
メロスには政治がわからぬ。
メロスは、村の牧人である。
笛を吹き、羊と遊んで暮して来た。
けれども邪悪に対しては、人一倍に敏感であった。
"""

## 30. 動詞
文章textに含まれる動詞をすべて表示せよ。

In [ ]:
import MeCab

text = """
メロスは激怒した。
必ず、かの邪智暴虐の王を除かなければならぬと決意した。
メロスには政治がわからぬ。
メロスは、村の牧人である。
笛を吹き、羊と遊んで暮して来た。
けれども邪悪に対しては、人一倍に敏感であった。
"""

tagger = MeCab.Tagger()
node = tagger.parse(text)
verbs = []

for line in node.split('\n'):
    if line == 'EOS' or line == '':
        continue
    try:
        surface, feature = line.split('\t')
        features = feature.split(',')
        if features[0] == '動詞' and features[6] != '*':
            base_form = features[6]
            verbs.append(base_form)
    except ValueError:
        continue

header = "表層形\t品詞,品詞細分類1,品詞細分類2,品詞細分類3,活用形,活用型,原形,読み,発音"
print("動詞のリスト")
print(verbs)

print("MeCabの解析結果")
print(header)
print(node)

## 31. 動詞の原型
文章textに含まれる動詞と、その原型をすべて表示せよ。

In [ ]:
import MeCab

text = """
メロスは激怒した。
必ず、かの邪智暴虐の王を除かなければならぬと決意した。
メロスには政治がわからぬ。
メロスは、村の牧人である。
笛を吹き、羊と遊んで暮して来た。
けれども邪悪に対しては、人一倍に敏感であった。
"""

tagger = MeCab.Tagger()
parsed = tagger.parse(text)
verb_base_forms = []

for line in parsed.split('\n'):
    if line == 'EOS' or line == '':
        continue
    try:
        surface, feature = line.split('\t')
        features = feature.split(',')
        if features[0] == '動詞':
            base_form = features[6]
            verb_base_forms.append((surface, base_form))
    except ValueError:
        continue

print("動詞とその原形一覧")
for surface, base in verb_base_forms:
    print(f"{surface} → {base}")

## 32. 「AのB」
文章textにおいて、2つの名詞が「の」で連結されている名詞句をすべて抽出せよ。



In [ ]:
import MeCab

text="""
メロスは激怒した。
必ず、かの邪智暴虐の王を除かなければならぬと決意した。
メロスには政治がわからぬ。
メロスは、村の牧人である。
笛を吹き、羊と遊んで暮して来た。
けれども邪悪に対しては、人一倍に敏感であった。
"""

tagger=MeCab.Tagger()
parsed_lines=tagger.parse(text).split('\n')

noun_phrases=[]
prev_noun=None
prev_no=False

for line in parsed_lines:
    if line=='EOS' or line =='':
        continue
    try:
        surface, feature=line.split('\t')
        features=feature.split(',')
        if features[0]=='名詞':
            if prev_no and prev_noun:
                noun_phrases.append(prev_noun+'の'+surface)
            prev_noun = surface
            prev_no = False
        elif surface=='の':
            prev_no=True
        else:
            prev_noun=None
            prev_no=False
    except ValueError:
        continue

for phrase in noun_phrases:
    print(phrase)

## 33. 係り受け解析
文章textに係り受け解析を適用し、係り元と係り先のトークン（形態素や文節などの単位）をタブ区切り形式ですべて抽出せよ。

In [ ]:
import CaboCha

text = "メロスは激怒した。"
c = CaboCha.Parser()
tree = c.parse(text)

tokens = []
token = tree.token(0)

while token:
    tokens.append(token)
    token = token.next

for token in tokens:
    if token.chunk:
        from_text = token.surface
        to_chunk = token.chunk.link
        if to_chunk != -1:
            to_text = tokens[tree.chunk(to_chunk).token_pos].surface
            print(f"{from_text}\t{to_text}")

## 34. 主述の関係
文章textにおいて、「メロス」が主語であるときの述語を抽出せよ。